# Problema 3

Este notebook contiene la solución explicada del siguiente problema:


Sea Graph2 una clase hija de Graph (implementación de grafo basada en
diccionario). Implementa una versión iterativa del método dfs para obtener el
recorrido en profundidad de un grafo desde un vértice concreto. 

Pista: Se recomienda usar una estructura
de pila para almacenar los nodos que se van visitando en cada camino.

La clase AdjacentVertex representa una tupla donde el primer elemento es un vértice y el segundo el peso asociado. 



In [22]:
class AdjacentVertex:
    """ This class allows us to represent a tuple
    with an adjacent vertex
    and the weight associated (by default None, for non-unweighted graphs)"""
    def __init__(self, vertex: object, weight: int = 1) -> None:
        self.vertex = vertex
        self.weight = weight

    def __eq_(self, other: 'AdjacentVertex') -> bool:
        if other is None: 
            return False
        return self.vertex == other.vertex and self.weight == other.weight 
        
    def __str__(self) -> str:
        """ returns the tuple (vertex, weight)"""
        if self.weight is not None:
            return '(' + str(self.vertex) + ',' + str(self.weight) + ')'
        else:
            return str(self.vertex)

In [23]:

class Graph:
    def __init__(self, vertices: list, directed: bool = True) -> None:
        """ We use a dictionary to represent the graph
        the dictionary's keys are the vertices
        The value associated for a given key will be the list of their neighbours.
        Initially, the list of neighbours is empty"""
        self._vertices = {}
        for v in vertices:
            self._vertices[v] = []
        self._directed = directed

    def add_vertex(self, vertex: str) -> None:
        if vertex in self._vertices.keys():
            print(vertex, ' already exists!')
            return
        self._vertices[vertex] = []

    def add_edge(self, start: object, end: object, weight: int = 1) -> None:
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return

        # adds to the end of the list of neighbours for start
        self._vertices[start].append(AdjacentVertex(end, weight))

        if not self._directed:
            # adds to the end of the list of neighbors for end
            self._vertices[end].append(AdjacentVertex(start, weight))

    def contain_edge(self, start: object, end: object) -> int:
        """ checks if the edge (start, end) exits. It does
        not exist return 0, eoc returns its weight or 1 (for unweighted graphs)"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return 0
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return 0

        # we search the AdjacentVertex whose v is equal to end

        for adj in self._vertices[start]:
            if adj.vertex == end:
                return adj.weight

        return 0  # does not exist

    def remove_edge(self, start: object, end: object):
        """ removes the edge (start, end)"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return

        # we must look for the adjacent AdjacentVertex (neighbour)  whose vertex is end, and then remove it
        exist = False
        for adj in self._vertices[start]:
            if adj.vertex == end:
                exist = True
                self._vertices[start].remove(adj)
        if not self._directed:
            # we must also look for the AdjacentVertex (neighbour)  whose vertex is end, and then remove it
            for adj in self._vertices[end]:
                if adj.vertex == start:
                    self._vertices[end].remove(adj)

        if not exist: 
            print("({},{}) does not exist!!!!".format(start, end))


    def __str__(self) -> str:
        """ returns a string containing the graph"""
        result = ''
        for v in self._vertices:
            result += '\n'+str(v)+':'
            for adj in self._vertices[v]:
                result += str(adj)+"  "
        result += '\n'
        return result

    def get_adjacent_vertices(self, start: object) -> list:
        """ returns a Python list containing the adjacent
        vertices of vertex. The list only contains the vertices"""
        if start not in self._vertices.keys():
            print(start, ' does not exist!')
            return None
        
        result = []
        for adj in self._vertices[start]:
            result.append(adj.vertex)
        return result


    def get_origins(self, end: object) -> list:
        """ returns a Python list containing those vertices that have
        an edge to vertex. The list is formed with objects of AdjacentVertex"""
        if end not in self._vertices.keys():
            print(end, ' does not exist!')
            return None
        
        result = []
        for v in self._vertices.keys():
            for adj in self._vertices[v]:
                if adj.vertex == end:
                    result.append(v)

        return result
  


Vamos a extender Graph para que contenga los recorridos dfs y bfs:

In [24]:
from queue import Queue
from queue import LifoQueue

class Graph2(Graph):

    def bfs(self, start: object) -> list:
        result = []

        visited = {}
        for v in self._vertices.keys():
            visited[v] = False
        q = Queue()
        q.put(start)
        visited[start] = True
        while not q.empty():
            v = q.get()
            result.append(v)
            lst_adj = self.get_adjacent_vertices(v)
            for v in lst_adj:
                if not visited[v]:
                    visited[v] = True
                    q.put(v)

        return result

    
    

    def dfs(self, start: object) -> list:

        result = []

        visited = {}
        for v in self._vertices.keys():
            visited[v] = False
        
        self._dfs(start, visited, result)
        return result


    
    def _dfs(self, start: object, visited: dict, result: list) -> None:
        visited[start] = True
        result.append(start)
        for v in self.get_adjacent_vertices(start):
            if not visited[v]:
                self._dfs(v, visited, result)
            

    def dfs_ite(self, start: object) -> list:

        if start not in self._vertices.keys():
            print(start, ' does not exist!!!')
            return None

        # create the list that will save the DFS traversal from start
        result = []

        # we create a dictionary to know what vertices have been already visited
        # At the beginning, all the vertices must have value False (not visited)
        visited = dict.fromkeys(self._vertices.keys(), False)
        
        # we will use a list (which will be used as a stack) to save the previous
        # visited vertices in the path. 

        stack = []
        stack.append(start)

        while len(stack)>0:
            s = stack.pop() # returns the last element added to the stack

            if not visited[s]:
                # we must visite it
                result.append(s)
                visited[s] = True
                # we must also get its adjacent vertices. We must add to the stack those adjacent vertices
                # that are not visited yet. 
                # The function reversed returns the reverse list of the input list.
                # This helps us to obtain the same dfs traversal that the traversal provided by dfs version recursive 
                # If we do not use reversed, the last element in the stack would be its last adjacent, so we would visit it as the first
                for v in reversed(self.get_adjacent_vertices(s)):
                    if not visited[v]:
                        stack.append(v)

        return result


<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/f/fe/Tred-G.svg/440px-Tred-G.svg.png' width='300'>


In [25]:
labels = ['a', 'b', 'c', 'd', 'e']
g = Graph2(labels)

g.add_edge('a','b')
g.add_edge('a','c')
g.add_edge('a','d')
g.add_edge('a','e')

g.add_edge('b','d')
g.add_edge('c','d')
g.add_edge('c','e')
g.add_edge('d','e')
# print(g)

g.add_vertex('f')
g.add_vertex('g')
g.add_edge('f','g')
print(g)



for v in g._vertices.keys():
    # print("Bfs traversal for {}: {}".format(v, g.bfs(v)))
    print("dfs traversal for {}: {}".format(v, g.dfs(v)))
    print("dfs based on stack traversal for {}: {}".format(v, g.dfs_ite(v)))
    break

    print()

# print("Bfs traversal full: {}".format(g.bfs_full()))
# print("dfs traversal full: {}".format(g.dfs_full()))



a:(b,1)  (c,1)  (d,1)  (e,1)  
b:(d,1)  
c:(d,1)  (e,1)  
d:(e,1)  
e:
f:(g,1)  
g:

dfs traversal for a: ['a', 'b', 'd', 'e', 'c']
dfs based on stack traversal for a: ['a', 'b', 'd', 'e', 'c']


<img src='https://static.javatpoint.com/ds/images/breadth-first-search-algorithm-example.png' widht='300'>

In [26]:
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
g = Graph2(labels)

g.add_edge('A','B')
g.add_edge('A','D')

g.add_edge('B','C')
g.add_edge('B','F')

g.add_edge('C','E')
g.add_edge('C','G')

g.add_edge('D','F')

g.add_edge('E','F')
g.add_edge('E','B')

g.add_edge('F','A')

g.add_edge('G','E')

g.add_vertex('H')
g.add_vertex('I')
g.add_vertex('J')

g.add_edge('H', 'I')
g.add_edge('H', 'J')


print(g)

for v in g._vertices.keys():
    print("Bfs traversal for {}: {}".format(v, g.bfs(v)))
    print("dfs traversal for {}: {}".format(v, g.dfs(v)))
    print("dfs traversal iterative for {}: {}".format(v, g.dfs_ite(v)))

    print()



A:(B,1)  (D,1)  
B:(C,1)  (F,1)  
C:(E,1)  (G,1)  
D:(F,1)  
E:(F,1)  (B,1)  
F:(A,1)  
G:(E,1)  
H:(I,1)  (J,1)  
I:
J:

Bfs traversal for A: ['A', 'B', 'D', 'C', 'F', 'E', 'G']
dfs traversal for A: ['A', 'B', 'C', 'E', 'F', 'G', 'D']
dfs traversal iterative for A: ['A', 'B', 'C', 'E', 'F', 'G', 'D']

Bfs traversal for B: ['B', 'C', 'F', 'E', 'G', 'A', 'D']
dfs traversal for B: ['B', 'C', 'E', 'F', 'A', 'D', 'G']
dfs traversal iterative for B: ['B', 'C', 'E', 'F', 'A', 'D', 'G']

Bfs traversal for C: ['C', 'E', 'G', 'F', 'B', 'A', 'D']
dfs traversal for C: ['C', 'E', 'F', 'A', 'B', 'D', 'G']
dfs traversal iterative for C: ['C', 'E', 'F', 'A', 'B', 'D', 'G']

Bfs traversal for D: ['D', 'F', 'A', 'B', 'C', 'E', 'G']
dfs traversal for D: ['D', 'F', 'A', 'B', 'C', 'E', 'G']
dfs traversal iterative for D: ['D', 'F', 'A', 'B', 'C', 'E', 'G']

Bfs traversal for E: ['E', 'F', 'B', 'A', 'C', 'D', 'G']
dfs traversal for E: ['E', 'F', 'A', 'B', 'C', 'G', 'D']
dfs traversal iterative for E: ['E'

# Ejercicio: 
1) Crea el siguiente grafo no dirigido y obtén sus recorridos a partir de cada uno de sus vértices: 
<img src='https://graphonline.ru/tmp/saved/XR/XRmBpPjiLYfUvjSa.png'>
2) Modifica el grafo anterior para que quede como el siguiente y obtén sus recorridos a partir de cada uno de sus vértices: 
<img src='https://graphonline.ru/tmp/saved/ep/eprzBsvpSiOpDvjj.png'>
3) Crea el siguiente grafo ponderado y y obtén sus recorridos a partir de cada uno de sus vértices:
<img src='https://infinitegraph.com/wp-content/uploads/2021/04/WeightedGraph01.png'>

4) Práctica con otros grafos. 